# Exploring citations with the Zotero API

In [ ]:
%%html
<iframe src="http://zotero.org/" width=700 height=350></iframe>

or: Never format your own bibliography again!
---------------------------------------------

The useful and fun things we will do today:

* Make a Zotero account (if you don't already have one)
* Install Zotero software from https://www.zotero.org/download/
* Add things to your Zotero library with the push of a button!
* Make a document, add references and citations without formatting a thing yourself.
* Make your own styles using a style editor such as http://editor.citationstyles.org/visualEditor/ 

And then, the fun things!

* Use Python to 'read' a Zotero library
* Pull the results into Graphviz and graph them!

This notebook includes the Python part of today's lesson.

Using Python to query Zotero
----------------------------

The first thing you need is the Zotero Python library. From a terminal command prompt, run

    pip install pyzotero
    
and then you're good to go.

In [ ]:
from pyzotero import zotero

Let's use Python to look through someone's Zotero library! For this example I am using the public Zotero group library "Digital Humanities". You can also do this with your own private libraries, or closed group libraries if you're a member. To use the Python interface at all, you will need to generate an API key at https://www.zotero.org/settings/keys. You'll plug that key in where I have indicated below. Essentially, this tells Zotero who is making the request and lets Zotero know whether you have permission to do the thing you're trying to do.

The Pyzotero API has some documentation here: http://pyzotero.readthedocs.org/en/latest/ 
although, like most technical documentation, it takes some getting used to. For now, we will retrieve some items from the "Digital Humanities" library and then take a look at one of them. 

In [ ]:
zotero_group = zotero.Zotero( 30, "group", 'SsbeUu6kJbK4w723P7GklmNb' )
things_in_library = zotero_group.items()
# See how many things we got back
print("We retrieved %d items" % len( things_in_library ))


So we got 100 things by asking for the group's `items`. Does that mean there are 100 things in the library?

In [ ]:
print("This group library has %d items in it" % zotero_group.num_items())

Hmm. So how come we only got 100?

In order to protect itself against obnoxiously huge requests, the Zotero server limits the amount of information it will give out in response to any one request. You can ask for any number you like at a time, up to 100, by saying `zotero_group.items( limit=100 )`, but if you want more than that you have to keep making new requests until you get them all. 

The way pyzotero handles larger libraries is a little intricate. One way to keep making new requests is to use the `.follow()` method - you can use this whenever you have just used a method that returns a list of items. `follow()` says "Okay, now give me the next chunk... and the next... and then next." If we already have 100 items, and know we have 556 more items to fetch, this means we need to run the method 6 times.

But as we're about to see, we have to figure out how to know when to stop.

In [ ]:
our_items = zotero_group.items( limit=100 ) 
for i in range(6):   # Run the 'follow' method six times
    our_items.extend( zotero_group.follow() )
    
print("We have %d items so far" % len( our_items ))

Well that's interesting - we saw before that the number of items in this library is supposedly 656, and now we have 700! 

That means that our first natural instinct for how to solve this problem - to divide num_items by 100 and figure out that way how many times we should iterate - won't work. What now? Where does this library end? Let's just keep trying a few more times. (Execute the code block below until you get a number that isn't an even multiple of 100.)

In [ ]:
our_items.extend( zotero_group.follow() )
    
print("We have %d items so far" % len( our_items ))

Okay - 885 is not a multiple of 100, so this is probably the end of the line. What would happen if we just tested, to see if there is any more?

In [ ]:
our_items.extend( zotero_group.follow() )
    
print("We have %d items so far" % len( our_items ))

Well! That's a nasty-looking error. And now we have the seemingly impossible situation that we have no way to know we're finished getting items out of Zotero until our code breaks on us.

As it happens, this sort of thing happens in Python all the time. We have to keep going until we get an error.

Crazy, you say? Ugly, you say? NO! Python expects you to try things until they fail, sometimes! I wasn't kidding about getting comfortable with failure.  Here's how we do it:

In [ ]:
our_items = zotero_group.items( limit=100 )
try:            # This says we anticipate that an error might crop up.
    while (1):  # This says, keep doing the following until something goes wrong.
        our_items.extend( zotero_group.follow() )
except StopIteration:         # This says, here is what we do if we get an error.
    pass        # This says, "ok thanks Python I was expecting that! Carry on."
    
print("We have %d items in total" % len( our_items ))

This is called "exception handling" and is very useful for when you know (or suspect) something will stop working.

So now we have all our items - let's see what the record looks like. For this we will print it out as JSON, so that we can use a JSON visualizer such as http://json.parser.online.fr.

In [ ]:
import json
print(json.dumps(our_items[1]))

Let's have a closer look at that `creators` field:

In [ ]:
print(our_items[1]['data']['creators'])

So we get their names, and we get the info of whether they are authors or editors or translators or what. Let's make a graph and see who publishes about digital humanities!

First, just to make sure we have the hang of this, let's list all the creators we find.

In [ ]:
for item in our_items:
    for creator in item['data']['creators']:
        print("%s %s was a(n) %s" % 
              ( creator['firstName'], 
               creator['lastName'], 
               creator['creatorType'] ))

Ooh huh, that's an ugly error. The `KeyError` means that we found a record that doesn't have a `firstName`. We could use a `try/except` block like we did above, but we could also just check whether each record has a creator before we try to list them. Let's see what's going on with these records by just printing out the whole structure when we encounter one. 

In [ ]:
for item in our_items:
    for creator in item['data']['creators']:
        if 'firstName' in creator:
            print("%s %s was a(n) %s" % ( 
                    creator['firstName'], 
                    creator['lastName'], 
                    creator['creatorType'] ))
        else:
            print(creator)

Well we can see what's happening, especially if we look at the raw record: It seems that names can come in the form `firstName` / `lastName`, or they can simply come in the form `name`, which is handy for institutions and people who don't have Western-style names. Either way, we need the full name in a single string, so we can use either 
"`firstName` `lastName`" or we can just use `name`.

But look, we ran into, another key error! There is an item without a `creator`. While we're at it, then, let's have another look at those.

In [ ]:
for item in our_items:
    if 'creators' in item['data']:
        for creator in item['data']['creators']:
            if( 'firstName' in creator ): # Print the name as firstName lastName
                print("%s %s was a(n) %s" % ( creator['firstName'], creator['lastName'], creator['creatorType'] ))
            else:                         # Just print the 'name'
                print("%s was a(n) %s" % ( creator['name'], creator['creatorType'] ))
    else:
        print(item)

Now we are error-free, so let's have a look at our creator-less items. It looks like they have an `itemType` of `attachment`, and they have this `parentItem` thing. It suggests they aren't independent records, but attachments that have been stored with some "main" bibliographic item - this, in turn, suggests the answer to our mystery of 885 items in a 656-item library! 

Since we are making a network graph of the creators, we can just skip any item that doesn't have a `creator` field.

In [ ]:
for item in our_items:
    if( 'creators' not in item['data'] ):
        continue  # Don't bother with this item
    for creator in item['data']['creators']:
        full_name = ''
        if( 'firstName' in creator ):
            full_name = ' '.join([creator['firstName'], 
                                  creator['lastName']])
        else:
            full_name = creator['name']
        print("%s was a(n) %s" % ( full_name, creator['creatorType'] ))

There we are! We can list our creators, which means that as soon as we have learned about graphs, we can graph them! Stay tuned.